## 月次集計(タイムゾーンが JST, Asia/Taipei の SSP)

※第一営業日目に回す

In [4]:
import pandas as pd
import numpy as np

import os
import pathlib
import zipfile

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

#from src import impala_config_hc2 as h2
#from src import impala_config_hc3 as h3

from ailab_tools.magics import slack_notify

In [5]:
#Gsheet URL
url = "https://docs.google.com/spreadsheets/d/1S_-N_caMj6iMOPNJW8WwwYQV18-eoXEHTwv-YR3DLyM/edit#gid=202356699"

In [ ]:
#シート上のデータ消去
for i in range(1,32):
    gs = GSheet()
    gs.clear_sheets(url, [i])

1.Google (JST, JPY)　SMN日本 (JST, JPY)

In [6]:
query = (
    f"""
    SELECT
      year,
      month,
      day,
      "OK"     AS "Result",
      COUNT(1) AS "Imp",
      SUM(CAST(coalesce(
              original_winning_price,
              winning_price) AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver
    WHERE
      year         = cast(year(add_months(now(), -1)) as string)
        AND
      month        = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id       = 1
        AND
      action_type  = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      dsp_company_id = 1
    GROUP BY year, month, day

    UNION ALL

    SELECT
      year,
      month,
      day,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(CAST(winning_price AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver_ilg
    WHERE
      year        = cast(year(add_months(now(), -1)) as string)
        AND
      month       = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id      = 1
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      dsp_company_id = 1
    GROUP BY year, month, day;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    gs = GSheet()
    gs.add_data(url, 17, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

SMN台湾 (Asia/Taipei, JPY)

In [7]:
query = (
    f"""
    SELECT
      year_local,
      month_local,
      day_local,
      "OK" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000000 AS "Spend (JPY)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'Asia/Taipei')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'Asia/Taipei')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'Asia/Taipei')) as day_local,
        CAST(coalesce(
              original_winning_price,
              winning_price) AS bigint) AS spend
      FROM dsplog.adserver
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 1
        AND
      action_type = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      dsp_company_id = 2
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local

    UNION ALL

    SELECT
      year_local,
      month_local,
      day_local,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000000 AS "Spend (JPY)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'Asia/Taipei')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'Asia/Taipei')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'Asia/Taipei')) as day_local,
        CAST(winning_price AS bigint) AS spend
      FROM dsplog.adserver_ilg
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 1
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      dsp_company_id = 2
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    gs = GSheet()
    gs.add_data(url, 18, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

 3.OpenX (JST, JPY)

In [8]:
query = (
    f"""
    SELECT
      year,
      month, 
      day,
      "OK"     AS "Result",
      COUNT(1) AS "Imp",
      SUM(CAST(coalesce(
          original_winning_price,
          winning_price) AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver
    WHERE
      year         = cast(year(add_months(now(), -1)) as string)
        AND
      month        = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id       = 3
        AND
      action_type  = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day

    UNION ALL

    SELECT
      year,
      month, 
      day,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(CAST(winning_price AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver_ilg
    WHERE
      year        = cast(year(add_months(now(), -1)) as string)
        AND
      month       = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id      = 3
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    gs = GSheet()
    gs.add_data(url, 19, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

9.Xrost (JST, JPY)

In [9]:
query = (
    f"""
    SELECT
      year,
      month, 
      day,
      "OK"     AS "Result",
      COUNT(1) AS "Imp",
      SUM(CAST(coalesce(
          original_winning_price,
          winning_price) AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver
    WHERE
      year         = cast(year(add_months(now(), -1)) as string)
        AND
      month        = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id       = 9
        AND
      action_type  = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day

    UNION ALL

    SELECT
      year,
      month, 
      day,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(CAST(winning_price AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver_ilg
    WHERE
      year        = cast(year(add_months(now(), -1)) as string)
        AND
      month       = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id      = 9
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    gs = GSheet()
    gs.add_data(url, 26, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

12.AdFunnel (JST, JPY)

In [10]:
query = (
    f"""
    SELECT
      year,
      month, 
      day,
      "OK"     AS "Result",
      COUNT(1) AS "Imp",
      SUM(CAST(coalesce(
          original_winning_price,
          winning_price) AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver
    WHERE
      year         = cast(year(add_months(now(), -1)) as string)
        AND
      month        = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id       = 12
        AND
      action_type  = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day

    UNION ALL

    SELECT
      year,
      month, 
      day,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(CAST(winning_price AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver_ilg
    WHERE
      year        = cast(year(add_months(now(), -1)) as string)
        AND
      month       = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id      = 12
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    gs = GSheet()
    gs.add_data(url, 2, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

13.COMPASS (JST, JPY)

In [11]:
query = (
    f"""
    SELECT
      year,
      month, 
      day,
      "OK"     AS "Result",
      COUNT(1) AS "Imp",
      SUM(CAST(coalesce(
          original_winning_price,
          winning_price) AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver
    WHERE
      year         = cast(year(add_months(now(), -1)) as string)
        AND
      month        = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id       = 13
        AND
      action_type  = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day

    UNION ALL

    SELECT
      year,
      month, 
      day,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(CAST(winning_price AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver_ilg
    WHERE
      year        = cast(year(add_months(now(), -1)) as string)
        AND
      month       = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id      = 13
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    gs = GSheet()
    gs.add_data(url, 12, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

14.AdStir (JST, JPY)

In [12]:
query = (
    f"""
    SELECT
      year,
      month, 
      day,
      "OK"     AS "Result",
      COUNT(1) AS "Imp",
      SUM(CAST(coalesce(
          original_winning_price,
          winning_price) AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver
    WHERE
      year         = cast(year(add_months(now(), -1)) as string)
        AND
      month        = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id       = 14
        AND
      action_type  = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day

    UNION ALL

    SELECT
      year,
      month,
      day,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(CAST(winning_price AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver_ilg
    WHERE
      year        = cast(year(add_months(now(), -1)) as string)
        AND
      month       = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id      = 14
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    gs = GSheet()
    gs.add_data(url, 4, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

15.BidSwitch YieldOne (JST, JPY)

In [13]:
query = (
    f"""
    SELECT
      year,
      month, 
      day,
      "OK"     AS "Result",
      COUNT(1) AS "Imp",
      SUM(CAST(coalesce(
          original_winning_price,
          winning_price) AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver
    WHERE
      year         = cast(year(add_months(now(), -1)) as string)
        AND
      month        = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id       = 15
        AND
      action_type  = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
       AND
     CAST(original_winning_price as bigint) > 0
    GROUP BY year, month, day

    UNION ALL

    SELECT
      year,
      month, 
      day,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(CAST(winning_price AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver_ilg
    WHERE
      year        = cast(year(add_months(now(), -1)) as string)
        AND
      month       = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id      = 15
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
     CAST(winning_price as bigint) > 0
    GROUP BY year, month, day;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    gs = GSheet()
    gs.add_data(url, 10, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

17.AdGeneration (JST, JPY)

In [14]:
query = (
    f"""
    SELECT
      year,
      month, 
      day,
      "OK"     AS "Result",
      COUNT(1) AS "Imp",
      SUM(CAST(coalesce(
          original_winning_price,
          winning_price) AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver
    WHERE
      year         = cast(year(add_months(now(), -1)) as string)
        AND
      month        = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id       = 17
        AND
      action_type  = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day

    UNION ALL

    SELECT
      year,
      month, 
      day,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(CAST(winning_price AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver_ilg
    WHERE
      year        = cast(year(add_months(now(), -1)) as string)
        AND
      month       = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id      = 17
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    gs = GSheet()
    gs.add_data(url, 3, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

19.AOL (JST, JPY)

In [15]:
query = (
    f"""
    SELECT
      year,
      month, 
      day,
      "OK"     AS "Result",
      COUNT(1) AS "Imp",
      SUM(CAST(coalesce(
          original_winning_price,
          winning_price) AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver
    WHERE
      year         = cast(year(add_months(now(), -1)) as string)
        AND
      month        = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id       = 19
        AND
      action_type  = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day

    UNION ALL

    SELECT
      year,
      month, 
      day,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(CAST(winning_price AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver_ilg
    WHERE
      year        = cast(year(add_months(now(), -1)) as string)
        AND
      month       = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id      = 19
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    gs = GSheet()
    gs.add_data(url, 1, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

20.Fluct (JST, JPY)

In [16]:
query = (
    f"""
    SELECT
      year,
      month, 
      day,
      "OK"     AS "Result",
      COUNT(1) AS "Imp",
      SUM(CAST(coalesce(
          original_winning_price,
          winning_price) AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver
    WHERE
      year         = cast(year(add_months(now(), -1)) as string)
        AND
      month        = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id       = 20
        AND
      action_type  = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day

    UNION ALL

    SELECT
      year,
      month, 
      day,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(CAST(winning_price AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver_ilg
    WHERE
      year        = cast(year(add_months(now(), -1)) as string)
        AND
      month       = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id      = 20
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    gs = GSheet()
    gs.add_data(url, 15, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

21.D2C (JST, JPY)

In [17]:
query = (
    f"""
    SELECT
      year,
      month,
      day,
      "OK"     AS "Result",
      COUNT(1) AS "Imp",
      SUM(CAST(coalesce(
          original_winning_price,
          winning_price) AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver
    WHERE
      year         = cast(year(add_months(now(), -1)) as string)
        AND
      month        = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id       = 21
        AND
      action_type  = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day

    UNION ALL

    SELECT
      year,
      month,
      day,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(CAST(winning_price AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver_ilg
    WHERE
      year        = cast(year(add_months(now(), -1)) as string)
        AND
      month       = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id      = 21
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    gs = GSheet()
    gs.add_data(url, 13, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

25.Taboola (JST, JPY)

In [18]:
query = (
    f"""
    SELECT
      year,
      month,
      day,
      "OK"     AS "Result",
      COUNT(1) AS "Imp",
      SUM(CAST(coalesce(
              original_winning_price,
              winning_price) AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver
    WHERE
      year         = cast(year(add_months(now(), -1)) as string)
        AND
      month        = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id       = 25
        AND
      action_type  = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      dsp_company_id = 1
    GROUP BY year, month, day

    UNION ALL

    SELECT
      year,
      month,
      day,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(CAST(winning_price AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver_ilg
    WHERE
      year        = cast(year(add_months(now(), -1)) as string)
        AND
      month       = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id      = 25
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      dsp_company_id = 1
    GROUP BY year, month, day;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    gs = GSheet()
    gs.add_data(url, 23, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

27.Profit X (JST, JPY)

In [19]:
query = (
    f"""
    SELECT
      year,
      month,
      day,
      "OK"     AS "Result",
      COUNT(1) AS "Imp",
      SUM(CAST(coalesce(
              original_winning_price,
              winning_price) AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver
    WHERE
      year         = cast(year(add_months(now(), -1)) as string)
        AND
      month        = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id       = 27
        AND
      action_type  = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      dsp_company_id = 1
    GROUP BY year, month, day

    UNION ALL

    SELECT
      year,
      month,
      day,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(CAST(winning_price AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver_ilg
    WHERE
      year        = cast(year(add_months(now(), -1)) as string)
        AND
      month       = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id      = 27
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      dsp_company_id = 1
    GROUP BY year, month, day;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    gs = GSheet()
    gs.add_data(url, 20, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

20.Fluct Native (JST, JPY)

In [20]:
query = (
    f"""
    SELECT
      year,
      month,
      day,
      "OK"     AS "Result",
      COUNT(1) AS "Imp",
      SUM(CAST(coalesce(
          original_winning_price,
          winning_price) AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver
    WHERE
      year         = cast(year(add_months(now(), -1)) as string)
        AND
      month        = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id       = 20
        AND
      action_type  = "I"
        AND
      creative_type IN (5, 6)
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day

    UNION ALL

    SELECT
      year,
      month,
      day,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(CAST(winning_price AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver_ilg
    WHERE
      year        = cast(year(add_months(now(), -1)) as string)
        AND
      month       = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id      = 20
        AND
      action_type = "I"
        AND
      creative_type IN (5, 6)
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    gs = GSheet()
    gs.add_data(url, 16, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

21.D2C Native (JST, JPY)

In [21]:
query = (
    f"""
    SELECT
      year,
      month,
      day,
      "OK"     AS "Result",
      COUNT(1) AS "Imp",
      SUM(CAST(coalesce(
          original_winning_price,
          winning_price) AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver
    WHERE
      year         = cast(year(add_months(now(), -1)) as string)
        AND
      month        = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id       = 21
        AND
      action_type  = "I"
        AND
      creative_type IN (5, 6)
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day

    UNION ALL

    SELECT
      year,
      month,
      day,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(CAST(winning_price AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver_ilg
    WHERE
      year        = cast(year(add_months(now(), -1)) as string)
        AND
      month       = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id      = 21
        AND
      action_type = "I"
        AND
      creative_type IN (5, 6)
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    GROUP BY year, month, day;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    gs = GSheet()
    gs.add_data(url, 14, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

31.Daddy/LFP (JST, JPY)

In [22]:
query = (
    f"""
    SELECT
      year,
      month,
      day,
      "OK"     AS "Result",
      COUNT(1) AS "Imp",
      SUM(CAST(coalesce(
              original_winning_price,
              winning_price) AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver
    WHERE
      year         = cast(year(add_months(now(), -1)) as string)
        AND
      month        = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id       = 31
        AND
      action_type  = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      dsp_company_id = 1
    GROUP BY year, month, day

    UNION ALL

    SELECT
      year,
      month,
      day,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(CAST(winning_price AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver_ilg
    WHERE
      year        = cast(year(add_months(now(), -1)) as string)
        AND
      month       = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id      = 31
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      dsp_company_id = 1
    GROUP BY year, month, day;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    gs = GSheet()
    gs.add_data(url, 28, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

32.GMO (JST, JPY)　2019/1/30開始　3月回すとき日付注意

In [23]:
%%slack_notify @yusuke_akada

query = (
    f"""
    SELECT
      year,
      month,
      day,
      "OK"     AS "Result",
      COUNT(1) AS "Imp",
      SUM(CAST(coalesce(
              original_winning_price,
              winning_price) AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver
    WHERE
      year         = cast(year(add_months(now(), -1)) as string)
        AND
      month        = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        and 
      ssp_id       = 32
        AND
      action_type  = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      dsp_company_id = 1
    GROUP BY year, month, day

    UNION ALL

    SELECT
      year,
      month,
      day,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(CAST(winning_price AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver_ilg
    WHERE
      year        = cast(year(add_months(now(), -1)) as string)
        AND
      month       = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
       AND 
      ssp_id      = 32
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      dsp_company_id = 1
    GROUP BY year, month, day;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    gs = GSheet()
    gs.add_data(url, 29, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

33.Popin (JST, JPY)　2019/8/22開始

In [24]:
%%slack_notify @yusuke_akada

query = (
    f"""
    SELECT
      year,
      month,
      day,
      "OK"     AS "Result",
      COUNT(1) AS "Imp",
      SUM(CAST(coalesce(
              original_winning_price,
              winning_price) AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver
    WHERE
      year         = cast(year(add_months(now(), -1)) as string)
        AND
      month        = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id       = 34
        AND
      action_type  = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      dsp_company_id = 1
    GROUP BY year, month, day

    UNION ALL

    SELECT
      year,
      month,
      day,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(CAST(winning_price AS bigint)) / 1000000 AS "Spend (JPY)"
    FROM dsplog.adserver_ilg
    WHERE
      year        = cast(year(add_months(now(), -1)) as string)
        AND
      month       = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id      = 34
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      dsp_company_id = 1
    GROUP BY year, month, day;

    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    gs = GSheet()
    gs.add_data(url, 31, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

In [25]:
import datetime
import pathlib
import math
from dateutil.relativedelta import relativedelta

In [26]:
year = datetime.date.today().year
month = datetime.date.today().month
today = datetime.date.today()
pre_month = today - relativedelta(months=1)
pre_month = datetime.date.strftime(pre_month, '%m')

In [27]:
print(pre_month)

10


In [28]:
print(year)

2019


In [29]:
gm = GMail()

mail_body = (
    f"""

    みなさま、
    
    お疲れ様です。

    前月分の国内SSP実績（imp・spend）送ります。
    ご査証下さい。
    
    よろしくお願い致します。
    
    {url}
    
    """
)

atr = gm.send_gmail(subject=f'SSP spned実績集計 {year}/{pre_month} (国内SSP分)送ります', 
                    message_text=mail_body,
                    to_addrs=['lsakaguchi@so-netmedia.jp','smn_relation@so-netmedia.jp'],
                    cc_addrs=['dm@so-netmedia.jp','yusuke_akada@so-netmedia.jp'])

INFO:ailab_tools.utils.gsuite:created OAuth2 credentials.
INFO:ailab_tools.utils.gsuite:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:googleapiclient.discovery:URL being requested: GET https://www.googleapis.com/discovery/v1/apis/gmail/v1/rest
INFO:googleapiclient.discovery:URL being requested: POST https://www.googleapis.com/gmail/v1/users/me/messages/send?alt=json
